In [1]:
import os, gc
import numpy as np
# import numba
# numba.set_num_threads(12) # 12 = cores

In [2]:
# import pandas as pd
# pd.set_option('use_numba', True)
import polars as pd


In [3]:
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
import time,datetime
from tqdm import tqdm
# from multiprocessing import Pool as ThreadPool

### Config

In [5]:
data_dir = './' # ../input/amex-default-prediction
data_dir_1 = './' # ../input/amex-default-prediction

### Preparation

In [5]:
# # reading csv in chunks
# train_data = pd.DataFrame()
# with pd.read_csv(os.path.join(data_dir, 'train_data.csv'), chunksize=10**5) as reader:
#     for counter, chunk in enumerate(reader):
#         for column in chunk.columns:
#             if str(chunk[column].dtype) == "float64":
#                 chunk[column] = chunk[column].astype(np.float32)
#             if str(chunk[column].dtype) == "int64":
#                 chunk[column] = chunk[column].astype(np.int32)
#         train_data = pd.concat([train_data, chunk])
#         _ = gc.collect()
        


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: float32(185), float64(1), int8(2), object(2)
memory usage: 3.9+ GB


In [7]:
train_data['D_64'].unique()

array(['O', 'R', nan, 'U', '-1'], dtype=object)

In [8]:
def denoise(df):
    df['D_63'] = df['D_63'].apply(lambda t: {'CR':0, 'XZ':1, 'XM':2, 'CO':3, 'CL':4, 'XL':5}[t]).astype(np.int8)
    df['D_64'] = df['D_64'].apply(lambda t: {np.nan:-1, 'O':0, '-1':1, 'R':2, 'U':3}[t]).astype(np.int8)
#     df['D_64'] = df['D_64'].apply(lambda t: {None:-1, 'O':0, '-1':1, 'R':2, 'U':3}[t]).astype(np.int8)    
    for col in tqdm(df.columns):
        if col not in ['customer_ID','S_2','D_63','D_64']:
            df[col] = np.floor(df[col]*100)
            # Return the floor of the input, element-wise. 
            # np.floor(np.array([-1.7, -1.5, -0.2, 0.2, 1.5, 1.7, 2.0])) => array([-2., -2., -1.,  0.,  1.,  1.,  2.])
    return df


In [9]:
train_data = denoise(train_data)

100%|██████████| 190/190 [00:04<00:00, 45.86it/s]


In [10]:
train_data.to_parquet("train_data",  compression='gzip', index=False)

In [1]:
del train_data

In [15]:
# reading csv in chunks
# test_data = pd.DataFrame()
# with pd.read_csv(os.path.join(data_dir, 'test_data.csv'), chunksize=10**5) as reader:
#     for counter, chunk in enumerate(reader):
#         for column in chunk.columns:
#             if str(chunk[column].dtype) == "float64":
#                 chunk[column] = chunk[column].astype(np.float32)
#             if str(chunk[column].dtype) == "int64":
#                 chunk[column] = chunk[column].astype(np.int32)
#         test_data = pd.concat([test_data, chunk])
#         _ = gc.collect()

In [18]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11363762 entries, 0 to 11363761
Columns: 190 entries, customer_ID to D_145
dtypes: float32(185), float64(1), int8(2), object(2)
memory usage: 8.1+ GB


In [17]:
test_data = denoise(test_data)

100%|██████████| 190/190 [00:07<00:00, 24.01it/s]


In [19]:
test_data.to_parquet("test_data",  compression='gzip', index=False)

In [12]:
del test_data

In [ ]:
df_tr = pd.read_parquet(os.path.join(data_dir_1, 'train_data'))
df_te = pd.read_parquet(os.path.join(data_dir_1, 'test_data'))
df = pd.concat([df_tr,df_te], parallel = True,)
del df_tr, df_te
_ = gc.collect()
df.write_parquet(f'train_test_data', compression='gzip',)
del df
_ = gc.collect()

In [11]:
df.shape

(16895213, 190)

### Feature Engineering section

In [1]:
# dfs[0][dfs[0]['customer_ID'] == '0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a']['rank_P_2']

In [7]:
def one_hot_encoding(dff,cols,is_drop=True):
    # cols => ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
            # col, df[col].unique() => B_30, [  0. 200. 100.]
    dummies = (
        pd.get_dummies(
            dff.select(cols),
            columns = None
        )
    )            

    # dropping one_hot_encoded null values (pola-rs also encode null values of a column).
    _ = [dummies.drop_in_place(col) for col in dummies.columns if col[-4:] == 'null']
    
    dummies.columns = list(map(lambda x: 'oneHot_%s'%x, dummies.columns))        
    
        # dummies.columns => Index(['oneHot_B_30_0.0', 'oneHot_B_30_100.0', 'oneHot_B_30_200.0'], dtype='object')
        # dummies['oneHot_B_30_0.0'].unique() => [1 0]
        
    # 'concat' is better for joining 'dummies with df' rather than 'join'. 
    # 'join' is not preferred as there is no need to match over a column, for dummies.
    dff = pd.concat([dff,dummies], how = 'horizontal', parallel = True,)  
    if is_drop:
        dff = dff.drop(cols)
    return dff

def cat_feature(dff):
    # type(df) => <class 'pandas.core.frame.DataFrame'>
    one_hot_features = [col for col in df.columns if 'oneHot' in col]  
    if lastk is None:
        num_agg_df = dff.groupby("customer_ID").agg([
                            pd.col(one_hot_features).sum().suffix("_sum"),
                            pd.col(one_hot_features).mean().suffix("_mean"),
                            pd.col(one_hot_features).std().suffix("_std"),
                            pd.col(one_hot_features).last().suffix("_last"), 
                        ])
    else:
        num_agg_df = dff.groupby("customer_ID").agg([
                            pd.col(one_hot_features).sum().suffix("_sum"),
                            pd.col(one_hot_features).mean().suffix("_mean"),
                            pd.col(one_hot_features).std().suffix("_std"),
                        ])            
        # num_agg_df.columns =>
        # Index(['oneHot_B_30_0.0_mean', 'oneHot_B_30_0.0_std', 'oneHot_B_30_0.0_sum',
        #        'oneHot_B_30_0.0_last',        

    if lastk is None:
        # cat_features  => ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
        cat_agg_df = dff.groupby("customer_ID").agg([
                            pd.col(cat_features).last().suffix("_last"),
                            pd.col(cat_features).n_unique().suffix("_nunique"),
                        ])
        # nunique => Return counts of unique elements in category for a particular customer_ID.
    else:
        cat_agg_df = dff.groupby("customer_ID").agg([
                            pd.col(cat_features).n_unique().suffix("_nunique"),
                        ])        
        # cat_agg_df.columns =>
        # Index(['B_30_last', 'B_30_nunique', 'B_38_last', 'B_38_nunique', 'D_114_last',
        #         ...
        #         ...
        #        'D_68_nunique'],
        #       dtype='object')    
        
    count_agg_df = dff.groupby("customer_ID").agg([
                        pd.col(['S_2']).count().suffix("_count"),
                    ])    
        # S_2 (timestamp_column)
        # count_agg_df.columns => Index(['S_2_count'], dtype='object')    
    
    dff = num_agg_df.join(cat_agg_df, on="customer_ID", how="left", ).join(count_agg_df, on="customer_ID", how="left", )    
    print('cat feature shape after engineering', dff.shape )

    return dff

def num_feature(dff):
    if num_features[0][:5] == 'rank_': 
        # num_features => ['rank_P_2', 'rank_D_39', 'rank_B_1', 'rank_B_2', 'rank_R_1', 'rank_S_3',......, 'rank_D_144', 'rank_D_145']        
        dff = dff.groupby("customer_ID").agg([
                            pd.col(num_features).last().suffix("_last"), 
                        ])        
        # .agg(['last']) => keep last rank of each group.

    else:
        # num_features => ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3',......, 'D_144', 'D_145']
        # num_features[0], num_features[0][:5] => P_2, P_2         
        if lastk is None:
            dff = dff.groupby("customer_ID").agg([
                                pd.col(num_features).sum().suffix("_sum"),
                                pd.col(num_features).mean().suffix("_mean"),
                                pd.col(num_features).std().suffix("_std"),
                                pd.col(num_features).min().suffix("_min"),                
                                pd.col(num_features).max().suffix("_max"),                
                                pd.col(num_features).last().suffix("_last"), 
                            ])            
        else:
            dff = dff.groupby("customer_ID").agg([
                                pd.col(num_features).sum().suffix("_sum"),
                                pd.col(num_features).mean().suffix("_mean"),
                                pd.col(num_features).std().suffix("_std"),
                                pd.col(num_features).min().suffix("_min"),                
                                pd.col(num_features).max().suffix("_max"),                
                            ])            

    if num_features[0][:5] != 'rank_':
        numeric_cols = [col for col in dff.columns if col != 'customer_ID']
        # num_agg_df; returns 'dataframe with operated columns only'.
        dff = dff.with_columns([
                            pd.col(numeric_cols).apply(lambda x: x // 0.01),
                        ])   # e.g., mean 92.846153 convert 9284.0
            # 92.846153 // 0.01 => 9284.0, 92.846153 / 0.01 => 9284.6153
            
    print('num feature shape after engineering', dff.shape )

    return dff

def diff_feature(dff):
#     diff_num_features = [f'diff_{col}' for col in num_features]            
    
    dff =  dff[['customer_ID']+num_features].select([
               pd.col('customer_ID'),
               pd.col(num_features).diff().over('customer_ID').prefix('diff_'),
            ])                    
    # .diff() => difference of a DataFrame element compared with another element in the 'previous row' (default). 
    # .prefix() => column labels are prefixed.        
    
    num_cols = [col for col in dff.columns if col != 'customer_ID']    
        
    if lastk is None:        
        dff = dff.groupby("customer_ID").agg([
                            pd.col(num_cols).sum().suffix("_sum"),
                            pd.col(num_cols).mean().suffix("_mean"),
                            pd.col(num_cols).std().suffix("_std"),
                            pd.col(num_cols).min().suffix("_min"),                
                            pd.col(num_cols).max().suffix("_max"),                
                            pd.col(num_cols).last().suffix("_last"),            
                        ])        
        
    else:
        dff = dff.groupby("customer_ID").agg([
                            pd.col(num_cols).sum().suffix("_sum"),
                            pd.col(num_cols).mean().suffix("_mean"),
                            pd.col(num_cols).std().suffix("_std"),
                            pd.col(num_cols).min().suffix("_min"),                
                            pd.col(num_cols).max().suffix("_max"),       
                        ])        

    num_cols = [col for col in dff.columns if col != 'customer_ID']        
    dff = dff.with_columns([
                        pd.col(num_cols).apply(lambda x: x // 0.01),
                    ])        

    print('diff feature shape after engineering', dff.shape )

    return dff

In [ ]:
# # creates new column 'assign'. trick; remove '[' and ']' symbols.
# df = df.select( 
#         diff_ = pd.col(num_features).diff()
#     )

In [8]:
n_cpu = 12 # 16
transform = [['','rank_','ym_rank_'],[''],['']]
# transform = [['rank_','ym_rank_'],[''],['']] # for debug


for li, lastk in enumerate([None,3,6]):
    for prefix in transform[li]:
# #         df = pd.read_feather(f'./input/train.feather').append(pd.read_feather(f'./input/test.feather'), ignore_index=True).reset_index(drop=True)
        df = pd.read_parquet(os.path.join(data_dir_1, 'train_test_data'))
#         df = pd.read_parquet(os.path.join(data_dir_1, 'train_data')) # for debugging    

        all_cols = [c for c in list(df.columns) if c not in ['customer_ID', 'S_2']]
         # leaving 'customer_ID' and 'S_2'
        
        cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
        num_features = [col for col in all_cols if col not in cat_features]
          # num_features => ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', ...,'S_17', ..., 'D_142', 'D_143', 'D_144', 'D_145']
          #  D_136, D_137, D_138, D_142 etc. have NaN values.
        
        for col in [col for col in df.columns if 'S_' in col or 'P_' in col]:
            if col != 'S_2':
                # with_columns; returns whole dataframe instead of 'dataframe with operated columns only'.
                df = df.with_columns([
                            pd.col(col).fill_null(0),
                        ])                

        if lastk is not None:
            prefix = f'last{lastk}_' + prefix
            print('all df shape',df.shape)            
            df = df.with_columns(
                        rank = pd.col('S_2').rank().reverse().over('customer_ID') 
                        # .reverse; current date have low rank
                    ).filter(pd.col('rank') < lastk).drop('rank') # keep top 3 rank or top 6 rank.           
            
            print(f'last {lastk} shape',df.shape)
                    
        if prefix == 'rank_':           
            # rank elements in each group. before and after no change in row count.
            df = df[['customer_ID']+num_features].select([
                        pd.col('customer_ID'),                
                       (pd.col(num_features).rank(method='average').over('customer_ID') / pd.col(num_features).count().over('customer_ID')).prefix('rank_'),
                    ])
                # .rank(pct=True) => Computes percentage rank of values i.e., arrange the ranks between '0.' and '1.'.
                # e.g. => elements - 4, 4, 7, 7, 5, 6
                # elements (sort) - 4, 4, 5, 6, 7, 7
                # ranks    - 1, 2, 3, 4, 5, 6
                # ranks (again) - 1.5, 1.5, 3, 4, 5.5, 5.5 => ranks of repeated items is averaged.
                # ranks (pct) - 0.250000, 0.250000, 0.500000, 0.666667, 0.916667, 0.916667            
            num_features = [f'rank_{col}' for col in num_features]                
                
        if prefix == 'ym_rank_': # ym => year-month
            df = df.with_columns(
                        S_2 = pd.col(['S_2']).apply(lambda x:x[:7]) # pick year-month from date column 'S_2'.
                    )                        

            # '.over' behaves similar to 'groupby'.
            df = df[['customer_ID','S_2']+num_features].select([
                        pd.col('customer_ID'),
                        (pd.col(num_features).rank(method='average').over("S_2") / pd.col(num_features).count().over("S_2")).prefix('ym_rank_'),
                    ])            
            num_features = [f'ym_rank_{col}' for col in num_features]

        if prefix in ['','last3_']:
            df = one_hot_encoding(df,cat_features,False)        

#         vc = df['customer_ID'].value_counts(sort=False).select([
#                 pd.cumsum("counts")
#             ])        
            
#         batch_size = int(np.ceil(len(vc) / n_cpu))        
#             # np.ceil(np.array([-1.7, -0.2, 0.2, 1.7, 2.0])) => array([-1., -0.,  1.,  2.,  2.]) 
            
#         dfs = []
#         start = 0
#         # make list of dataframe batches in accordance to cpu cores.
#         for i in range(min(n_cpu,int(np.ceil(len(vc) / batch_size)))):                
#             vc_ = vc[i*batch_size:(i+1)*batch_size]
#                 # vc_ =>
#                 # df['customer_ID'].value_counts(sort=False).cumsum() => 
#                 # 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a         13
#                 # 00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5         26
#                 # ...
#                 # fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eaba8b115f71cab04681    5531451
#             dfs.append(df[start:vc_[-1].item()])
#                 # df[start:vc_[-1]] => df[0:5531451]
#             start = vc_[-1].item()

#         pool = ThreadPool(n_cpu)

        if prefix in ['','last3_']:
#             cat_feature_df = pd.concat(pool.map(cat_feature,tqdm(dfs,desc='cat_feature'))).reset_index(drop=True)
#             cat_feature_df.to_feather(f'./input/{prefix}cat_feature.feather')

            cat_feature_df = cat_feature(df)
#             cat_feature_df = pd.concat([cat_feature(dfs[l]) for l in range(len(dfs))], how = 'vertical', parallel = True,)
            cat_feature_df.write_parquet(f'{prefix}cat_feature', compression='gzip')
                # tqdm(dfs,desc='cat_feature') => desc; add message to tqdm progress.
                # pool.map(function_name, arguments)
            del cat_feature_df
            _ = gc.collect()

        if prefix in ['','last3_','last6_','rank_','ym_rank_']:
#             num_feature_df = pd.concat(pool.map(num_feature,tqdm(dfs,desc='num_feature'))).reset_index(drop=True)
#             num_feature_df.to_feather(f'./input/{prefix}num_feature.feather')

            num_feature_df = num_feature(df)
#             num_feature_df = pd.concat([num_feature(dfs[l]) for l in range(len(dfs))], how = 'vertical', parallel = True,)
            num_feature_df.write_parquet(f'{prefix}num_feature', compression='gzip')
            del num_feature_df
            _ = gc.collect()        

        if prefix in ['','last3_']:
#             diff_feature_df = pd.concat(pool.map(diff_feature,tqdm(dfs,desc='diff_feature'))).reset_index(drop=True)
#             diff_feature_df.to_feather(f'./input/{prefix}diff_feature.feather')

            diff_feature_df = diff_feature(df)
#             diff_feature_df = pd.concat([diff_feature(dfs[l]) for l in range(len(dfs))], how = 'vertical', parallel = True,)
            diff_feature_df.write_parquet(f'{prefix}diff_feature', compression='gzip') 
            del diff_feature_df
            _ = gc.collect()        

#         pool.close()


num feature shape after engineering (1383534, 178)
num feature shape after engineering (1383534, 1063)
all df shape (16895213, 190)
last 3 shape (2756121, 190)
cat feature shape after engineering (1383534, 148)
num feature shape after engineering (1383534, 886)
diff feature shape after engineering (1383534, 886)
all df shape (16895213, 190)
last 6 shape (6790883, 190)
num feature shape after engineering (1383534, 886)


In [ ]:
# keep copies of ['cat','num','diff','rank_num','last3_cat','last3_num','last3_diff', 'last6_num','ym_rank_num'] files to the inside of folder 'extra'.